In [1]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import tensorflow_recommenders as tfrs

In [2]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

Let's pretend we have 3 users:
1. Clarissa is a vegetarian who is lactose intolerant. 🥝🍅
2. Martin is an athlete who prefers meat dishes to vegetable dishes. 🍖🥩
3. Stella - has no special preferences, eats mostly everything. 🍲🍠

In [3]:
clarissa = {'id':'clr', 'prefers': ['vegetables', 'vegan', 'nomeat', 'vegetarian'], 'hates': ['meat', 'lactose']}
martin = {'id':'mrt', 'prefers': 'meat', 'hates': 'vegetables'}
stella = {'id':'stl', 'prefers': '', 'hates': ''}

We will additionally enter the factors Calories, Protein, Fat, Carbohydrates for our requirements. **(The numbers were taken at random and do not reflect proportions or recommendations - the numbers are just an example)**
* Clarissa - 2000/100/60/120
* Martin, 2600/150/100/328.
* Stella - 2200/80/50/100

In [4]:
def set_requirements(user, calories, proteins, fats, carbs):
  user['calories'] = calories
  user['proteins'] = proteins
  user['fats'] = fats
  user['carbs'] = carbs
set_requirements(clarissa, 2000, 100, 60, 120)
set_requirements(martin, 2600, 150, 100, 328)
set_requirements(martin, 2200, 80, 50, 100)


# Data processing 📊

Loading recipes and user_interactions. Nutritions showed like:calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat (PDV) , and carbohydrates (PDV)

In [5]:
raw_recipes = pd.read_csv('./sample_data/RAW_recipes.csv')
raw_interactions = pd.read_csv('./sample_data/RAW_interactions.csv')
raw_recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


Merge data interactions with recipe infos

In [6]:
interactions_with_recipe_info = pd.merge(raw_interactions, raw_recipes[['name', 'tags', 'ingredients', 'id']], left_on='recipe_id', right_on='id', how='left')
interactions_with_recipe_info = interactions_with_recipe_info[['user_id', 'recipe_id', 'rating', 'name', 'tags', 'ingredients']]
interactions_with_recipe_info.rename(columns={'name': 'recipe_name'}, inplace=True)
interactions_with_recipe_info.head()

,user_id,recipe_id,rating,recipe_name,tags,ingredients
0,38094,40893,4,white bean green chile pepper soup,"['weeknight', 'time-to-make', 'course', 'main-...","['great northern beans', 'yellow onion', 'dice..."
1,1293707,40893,5,white bean green chile pepper soup,"['weeknight', 'time-to-make', 'course', 'main-...","['great northern beans', 'yellow onion', 'dice..."
2,8937,44394,4,devilicious cookie cake delights,"['30-minutes-or-less', 'time-to-make', 'course...","[""devil's food cake mix"", 'vegetable oil', 'eg..."
3,126440,85009,5,baked potato toppings,"['15-minutes-or-less', 'time-to-make', 'course...","['mayonnaise', 'salsa', 'cheddar cheese', 'ref..."
4,57222,85009,5,baked potato toppings,"['15-minutes-or-less', 'time-to-make', 'course...","['mayonnaise', 'salsa', 'cheddar cheese', 'ref..."


In [7]:
import ast

def convert_to_list(data_str):
  try:
    return ast.literal_eval(data_str)
  except (SyntaxError, ValueError):
    return []

In [8]:
interactions_with_recipe_info['ingredients_str'] = interactions_with_recipe_info['ingredients'].apply(convert_to_list)
interactions_with_recipe_info['tags_str'] = interactions_with_recipe_info['tags'].apply(convert_to_list)

raw_recipes['ingredients_str'] = raw_recipes['ingredients'].apply(convert_to_list)
raw_recipes['tags_str'] = raw_recipes['tags'].apply(convert_to_list)

In [9]:
interactions_with_recipe_info['ingredients_str'].str.len().unique()

array([ 9,  4, 13,  2, 12, 10, 16,  8, 14, 15,  7, 11,  6,  5, 23, 22, 17,
        3, 18, 19, 21, 26, 29, 24, 20, 27, 25, 33, 30, 31,  1, 28, 35, 32,
       34, 43, 40, 39, 37, 36, 38], dtype=int64)

Formatting data for Datasets

In [10]:
interactions_with_recipe_info['user_id'] = interactions_with_recipe_info.user_id.astype("str")
interactions_with_recipe_info['recipe_id'] = interactions_with_recipe_info.recipe_id.astype("str")
interactions_with_recipe_info['rating'] = interactions_with_recipe_info.rating.astype(np.float32)
interactions_with_recipe_info['recipe_name'] = interactions_with_recipe_info.recipe_name.astype("str")
interactions_with_recipe_info['tags'] = interactions_with_recipe_info.tags.astype("str")
interactions_with_recipe_info['ingredients'] = interactions_with_recipe_info.ingredients.astype("str")


Creating train preferences for users

In [11]:
user_preferences = interactions_with_recipe_info[['user_id', 'rating', 'tags_str']]
user_preferences.head()

,user_id,rating,tags_str
0,38094,4.0,"[weeknight, time-to-make, course, main-ingredi..."
1,1293707,5.0,"[weeknight, time-to-make, course, main-ingredi..."
2,8937,4.0,"[30-minutes-or-less, time-to-make, course, mai..."
3,126440,5.0,"[15-minutes-or-less, time-to-make, course, mai..."
4,57222,5.0,"[15-minutes-or-less, time-to-make, course, mai..."


In [12]:
user_liked_tags = user_preferences[user_preferences['rating'] >= 3].groupby('user_id')['tags_str'].apply(list).reset_index()
user_unliked_tags = user_preferences[user_preferences['rating'] < 3].groupby('user_id')['tags_str'].apply(list).reset_index()
users_ds = pd.DataFrame(interactions_with_recipe_info['user_id'].unique(), columns=['user_id'])


In [13]:
users_ds = users_ds.merge(user_liked_tags, on='user_id', how='left')
users_ds = users_ds.rename(columns={'tags_str': 'liked_tags'})
users_ds = users_ds.merge(user_unliked_tags, on='user_id', how='left')
users_ds = users_ds.rename(columns={'tags_str': 'unliked_tags'})

In [14]:
users_ds.head()

,user_id,liked_tags,unliked_tags
0,38094,"[[weeknight, time-to-make, course, main-ingred...",NaN
1,1293707,"[[weeknight, time-to-make, course, main-ingred...","[[60-minutes-or-less, time-to-make, course, ma..."
2,8937,"[[30-minutes-or-less, time-to-make, course, ma...",NaN
3,126440,"[[15-minutes-or-less, time-to-make, course, ma...","[[60-minutes-or-less, time-to-make, course, pr..."
4,57222,"[[15-minutes-or-less, time-to-make, course, ma...","[[60-minutes-or-less, time-to-make, course, ma..."


Looking that some of users don't have unliked_tags, so replace all Nan with empty arrays

In [15]:
def unique_tags_in_list(tags_list):
  if isinstance(tags_list, list):
    unique_tags = []
    for sublist in tags_list:
      if isinstance(sublist, list):
        for tag in sublist:
          if tag not in unique_tags:
            unique_tags.append(tag)
    return unique_tags
  else:
    return []

users_ds['liked_tags'] = users_ds['liked_tags'].apply(unique_tags_in_list)
users_ds['unliked_tags'] = users_ds['unliked_tags'].apply(unique_tags_in_list)

In [16]:
def remove_duplicate_tags(row):
  liked_tags = set(row['liked_tags']) if isinstance(row['liked_tags'], list) else set()
  unliked_tags = set(row['unliked_tags']) if isinstance(row['unliked_tags'], list) else set()
  duplicate_tags = liked_tags.intersection(unliked_tags)
  row['liked_tags'] = [tag for tag in row['liked_tags'] if tag not in duplicate_tags] if isinstance(row['liked_tags'], list) else []
  row['unliked_tags'] = [tag for tag in row['unliked_tags'] if tag not in duplicate_tags] if isinstance(row['unliked_tags'], list) else []
  return row

users_ds = users_ds.apply(remove_duplicate_tags, axis=1)


In [17]:
users_ds.head()

,user_id,liked_tags,unliked_tags
0,38094,"[weeknight, time-to-make, course, main-ingredi...",[]
1,1293707,"[weeknight, soups-stews, beans, crock-pot-slow...","[cupcakes, finger-food, cakes, english, cake-f..."
2,8937,"[30-minutes-or-less, time-to-make, course, mai...",[]
3,126440,"[15-minutes-or-less, 3-steps-or-less, jewish-s...",[]
4,57222,"[condiments-etc, salads, beans, grains, south-...","[kwanzaa, dairy-free]"


In [18]:
interactions_with_recipe_info = pd.merge(interactions_with_recipe_info, users_ds, on='user_id', how='left')

Creating datasets

In [19]:
# Параметры
MAX_TAG_LENGTH = 30
PADDING_VALUE = "empty"

interactions_with_recipe_info['liked_tags'] = [
    (tags + [PADDING_VALUE] * (MAX_TAG_LENGTH - len(tags)))[:MAX_TAG_LENGTH] 
    for tags in interactions_with_recipe_info['liked_tags']
]
interactions_with_recipe_info['unliked_tags'] = [
    (tags + [PADDING_VALUE] * (MAX_TAG_LENGTH - len(tags)))[:MAX_TAG_LENGTH] 
    for tags in interactions_with_recipe_info['unliked_tags']
]
interactions_with_recipe_info['ingredients_str'] =  [
    (tags + [PADDING_VALUE] * (MAX_TAG_LENGTH - len(tags)))[:MAX_TAG_LENGTH] 
    for tags in interactions_with_recipe_info['ingredients_str'] ]

interactions_with_recipe_info['tags_str'] =  [
    (tags + [PADDING_VALUE] * (MAX_TAG_LENGTH - len(tags)))[:MAX_TAG_LENGTH] 
    for tags in interactions_with_recipe_info['tags_str'] ]

raw_recipes['tags_str'] = [
    (tags + [PADDING_VALUE] * (MAX_TAG_LENGTH - len(tags)))[:MAX_TAG_LENGTH] 
    for tags in raw_recipes['tags_str'] ]

raw_recipes['ingredients_str'] = [
    (tags + [PADDING_VALUE] * (MAX_TAG_LENGTH - len(tags)))[:MAX_TAG_LENGTH] 
    for tags in raw_recipes['ingredients_str'] ]

In [20]:
interactions_with_recipe_info['ingredients_str'].str.len().unique(), interactions_with_recipe_info['unliked_tags'].str.len().unique(), 

(array([30], dtype=int64), array([30], dtype=int64))

In [21]:
interactions_with_recipe_info['liked_tags_str'] = interactions_with_recipe_info['liked_tags'].apply(lambda x: ','.join(x))
interactions_with_recipe_info['unliked_tags_str'] = interactions_with_recipe_info['unliked_tags'].apply(lambda x: ','.join(x))
interactions_with_recipe_info['tags_str'] = interactions_with_recipe_info['tags_str'].apply(lambda x: ','.join(x))

In [22]:
def split_tags(tags_string):
    return tf.strings.split(tf.strings.strip(tags_string), ',')

In [23]:
tf.compat.v1.enable_eager_execution()

In [24]:
def squeeze_tags(data):
    return {
        "user_id": data["user_id"],
        "recipe_id": data["recipe_id"],
        "rating": data["rating"],
        "recipe_name": data["recipe_name"],
        "tags": tf.squeeze(data["tags"], axis=0),  # Укажите ось, которую нужно удалить
        "liked_tags": tf.squeeze(data["liked_tags"], axis=0),  # Укажите ось, которую нужно удалить
        "unliked_tags": tf.squeeze(data["unliked_tags"], axis=0)  # Укажите ось, которую нужно удалить
    }

ratings = tf.data.Dataset.from_tensor_slices((tf.cast(interactions_with_recipe_info['user_id'].values, tf.string),
                                              tf.cast(interactions_with_recipe_info['recipe_id'].values, tf.string),
                                              tf.cast(interactions_with_recipe_info['rating'].values, tf.float32),
                                              tf.cast(interactions_with_recipe_info['recipe_name'].values, tf.string),
                                              tf.cast(interactions_with_recipe_info['tags_str'].values.reshape(-1,1), tf.string),
                                              tf.cast(interactions_with_recipe_info['liked_tags_str'].values.reshape(-1,1), tf.string),
                                              tf.cast(interactions_with_recipe_info['unliked_tags_str'].values.reshape(-1,1), tf.string)
                                              )).map(lambda x, x1, x2, x3, x4, x5, x6: {
                                                  "user_id": x,
                                                  "recipe_id": x1,
                                                  "rating": x2,
                                                  "recipe_name": x3,
                                                  "tags": split_tags(x4),
                                                  "liked_tags": split_tags(x5),
                                                  "unliked_tags": split_tags(x6)
                                              }).map(squeeze_tags)


In [25]:
for data in ratings.take(1).as_numpy_iterator():
    print(data)

{'user_id': b'38094', 'recipe_id': b'40893', 'rating': 4.0, 'recipe_name': b'white bean   green chile pepper soup', 'tags': array([b'weeknight', b'time-to-make', b'course', b'main-ingredient',
       b'preparation', b'occasion', b'soups-stews', b'beans',
       b'vegetables', b'easy', b'crock-pot-slow-cooker', b'dietary',
       b'equipment', b'empty', b'empty', b'empty', b'empty', b'empty',
       b'empty', b'empty', b'empty', b'empty', b'empty', b'empty',
       b'empty', b'empty', b'empty', b'empty', b'empty', b'empty'],
      dtype=object), 'liked_tags': array([b'weeknight', b'time-to-make', b'course', b'main-ingredient',
       b'preparation', b'occasion', b'soups-stews', b'beans',
       b'vegetables', b'easy', b'crock-pot-slow-cooker', b'dietary',
       b'equipment', b'60-minutes-or-less', b'main-dish', b'fruit',
       b'vegan', b'vegetarian', b'stove-top', b'black-beans',
       b'30-minutes-or-less', b'low-protein', b'salads', b'dinner-party',
       b'low-cholesterol', b'lo

In [26]:
raw_recipes['ingredients_str'] = raw_recipes['ingredients_str'].apply(lambda x: ','.join(x))
raw_recipes['tags_str'] = raw_recipes['tags_str'].apply(lambda x: ','.join(x))

In [27]:
raw_recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,ingredients_str,tags_str
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,"winter squash,mexican seasoning,mixed spice,ho...","60-minutes-or-less,time-to-make,course,main-in..."
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,"prepared pizza crust,sausage patty,eggs,milk,s...","30-minutes-or-less,time-to-make,course,main-in..."
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,"ground beef,yellow onions,diced tomatoes,tomat...","time-to-make,course,preparation,main-dish,chil..."
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,"spreadable cheese with garlic and herbs,new po...","60-minutes-or-less,time-to-make,course,main-in..."
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,"tomato juice,apple cider vinegar,sugar,salt,pe...","weeknight,time-to-make,course,main-ingredient,..."


In [28]:
raw_recipes['id'] = raw_recipes.id.astype("str")
raw_recipes['name'] = raw_recipes.name.astype("str")
raw_recipes['tags'] = raw_recipes.tags.astype("str")
raw_recipes['ingredients'] = raw_recipes.ingredients.astype("str")

In [29]:
raw_recipes['name'][:1]

0    arriba   baked winter squash mexican style
Name: name, dtype: object

In [31]:
# prompt: Создай на основе raw_recipes датасет, но учти что ingredients_str это массив

recipes = tf.data.Dataset.from_tensor_slices((tf.cast(raw_recipes['id'].values, tf.string),
                                              tf.cast(raw_recipes['name'].values, tf.string),
                                              tf.cast(raw_recipes['tags_str'].values.reshape(-1, 1), tf.string)
                                            #   tf.cast(raw_recipes['ingredients_str'].values.reshape(-1, 1), tf.string),
                                              )
                                             ).map(lambda x, x1, x2: {
                                                  "recipe_id": x,
                                                  "recipe_name": x1,
                                                  "tags": split_tags(x2),
                                                #   "ingredients": split_tags(x3)
                                              }).map(lambda data: {
        **data,
        "tags": tf.squeeze(data["tags"], axis=0)  # Удаляем дополнительные измерения
        # "ingredients": tf.squeeze(data["ingredients"], axis=0)  # Удаляем дополнительные измерения
    })



In [32]:
sizes = []
for data in recipes.as_numpy_iterator():
  sizes.append(len(data["tags"]))
print(np.unique(sizes))  

[30]


In [33]:
recipe_names = tf.data.Dataset.from_tensor_slices((tf.cast(raw_recipes['name'].values.reshape(-1, 1), tf.string))).map(lambda x: x)

# Towers 🗼

For our towers set dimensionality of the query and candidate representations: **32**. Higher values will correspond to models that may be more accurate, but will also be slower to fit and more prone to overfitting.

In [34]:
embedding_dimension = 32

## User tower 👷

Lets start creating our towers with User towers. We will compute by:
1. User id
2. Prefers and hates

### User ID model

In [35]:
unique_user_ids = users_ds["user_id"].unique()
unique_user_ids[:10]

array(['38094', '1293707', '8937', '126440', '57222', '52282', '124416',
       '2000192946', '76535', '273745'], dtype=object)

In [36]:
user_id_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids)+1, embedding_dimension)
])

### User Prefers models

In [37]:
# prompt: модель для liked_tags которым стоит отдать предпочтение, но учти что liked_tags в датасете это массив

unique_liked_tags = set()
for tags in users_ds['liked_tags']:
  if isinstance(tags, list):
    for tag in tags:
      unique_liked_tags.add(tag)
unique_liked_tags = list(unique_liked_tags)

liked_tags_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_liked_tags, mask_token=None),
  tf.keras.layers.Embedding(len(unique_liked_tags) + 1, embedding_dimension)
])


In [38]:
unique_liked_tags[:10]

['',
 'whole-turkey',
 'puddings-and-mousses',
 'cocktails',
 'pumpkin-bread',
 'vegetables',
 'bear',
 'cuban',
 'saltwater-fish',
 'ham-and-bean-soup']

In [39]:

unique_unliked_tags = set()
for tags in users_ds['unliked_tags']:
  if isinstance(tags, list):
    for tag in tags:
      unique_unliked_tags.add(tag)
unique_unliked_tags = list(unique_unliked_tags)

unliked_tags_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_unliked_tags, mask_token=None),
  tf.keras.layers.Embedding(len(unique_unliked_tags) + 1, embedding_dimension)
])
unique_unliked_tags[:10]

['',
 'whole-turkey',
 'puddings-and-mousses',
 'cocktails',
 'pumpkin-bread',
 'vegetables',
 'bear',
 'cuban',
 'saltwater-fish',
 'freezer']

## Recipe tower 🌭

In [55]:
unique_recipe_names = np.unique(list(raw_recipes["name"].unique()))
recipe_name_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_recipe_names, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_recipe_names) + 1, embedding_dimension)
])
unique_recipe_names[:10]

<tf.Tensor: shape=(10, 32), dtype=float32, numpy=
array([[ 0.0472048 ,  0.01294236,  0.00443701,  0.03320727,  0.03093543,
        -0.00575296, -0.00971135, -0.00443405, -0.0112393 ,  0.04181773,
        -0.01955422, -0.02848119,  0.02421819,  0.00236275, -0.04367229,
        -0.0098153 ,  0.02960339, -0.02839004,  0.04600623, -0.00764652,
        -0.03071568,  0.04263257,  0.03541404,  0.00306255,  0.02307874,
        -0.03934962,  0.02299384,  0.02980052, -0.01855361, -0.04155486,
         0.00100814, -0.00252416],
       [ 0.01989014,  0.02301419, -0.03366714, -0.01993567, -0.02033699,
         0.02615819,  0.04491473,  0.01740885,  0.02885518,  0.04962257,
        -0.01450702, -0.03947305,  0.03339407, -0.00873897,  0.00290372,
        -0.00048428, -0.02043494, -0.02024843, -0.01525304,  0.00413046,
        -0.01673555,  0.0418126 ,  0.0454229 ,  0.04387082,  0.00041959,
        -0.0122963 , -0.00599637,  0.00350878, -0.01531995,  0.03534785,
        -0.0450451 , -0.04944437],
    

In [41]:
# prompt: напиши вызов recipe_name_model

recipe_name_model(['some recipe name'])


<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[-0.01512814,  0.00578652, -0.02536417,  0.01278904, -0.02616717,
         0.04773802, -0.02358906, -0.00427809, -0.00764314,  0.0321222 ,
        -0.0138072 ,  0.04330536, -0.02166371, -0.02522388, -0.02843325,
        -0.02910688, -0.04606888,  0.02071381, -0.04990273,  0.00213184,
         0.04562029, -0.03151394,  0.03627158, -0.0352541 ,  0.0396946 ,
        -0.0235576 , -0.01854372, -0.04658786, -0.01251549,  0.04925865,
        -0.02487599, -0.03450589]], dtype=float32)>

### Recipe ingredient&tag model

In [56]:
unique_tags = set()
for tags in raw_recipes['tags'].apply(convert_to_list):
  if isinstance(tags, list):
    for tag in tags:
      unique_tags.add(tag)
unique_tags = np.unique(list(unique_tags))

tags_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_tags, mask_token=None),
  tf.keras.layers.Embedding(len(unique_tags) + 1, embedding_dimension)
])
unique_tags[:10]

<tf.Tensor: shape=(10, 32), dtype=float32, numpy=
array([[ 4.87420000e-02,  4.77958806e-02, -8.03662464e-03,
        -2.93976795e-02, -4.12520058e-02,  1.25867613e-02,
        -1.12114921e-02,  4.56080474e-02, -4.02108915e-02,
         4.72535156e-02, -1.00573897e-02, -4.15225402e-02,
        -2.64863018e-02, -4.40023653e-02,  2.14552395e-02,
        -2.47882679e-03, -3.11393142e-02,  4.25396115e-03,
         2.97096111e-02,  4.14888933e-03, -5.62032312e-03,
         1.54796280e-02, -1.90889481e-02, -1.47879720e-02,
         2.11942680e-02,  1.29952095e-02,  3.20545323e-02,
         3.85669507e-02, -8.96073505e-03, -1.79054849e-02,
        -7.65532255e-03,  1.17259137e-02],
       [ 3.25552486e-02,  7.96619803e-03, -1.23373643e-02,
         2.05368660e-02, -4.82404009e-02,  1.09242797e-02,
         3.83428819e-02, -1.62575841e-02, -1.55601278e-02,
         3.20783295e-02,  1.02849007e-02,  4.92631532e-02,
        -6.01029396e-03, -3.65650170e-02, -3.61375436e-02,
        -5.90885803e-0

## Combine models

In [49]:
class UserModel(tf.keras.Model):
    def __init__(self, unique_user_ids, tags_model, embedding_dimension):
        super().__init__()
        
        # User ID embeddings
        self.user_id_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
        ])
        
        # Liked tags embeddings
        self.liked_tags_model = tf.keras.Sequential([
          tf.keras.layers.StringLookup(
              vocabulary=unique_tags, mask_token=None),
          tf.keras.layers.Embedding(len(unique_tags) + 1, embedding_dimension)
        ])

        # Unliked tags embeddings
        self.unliked_tags_model = tf.keras.Sequential([
              tf.keras.layers.StringLookup(
                  vocabulary=unique_tags, mask_token=None),
              tf.keras.layers.Embedding(len(unique_tags) + 1, embedding_dimension)
            ])

        # Penalty weight for unliked tags
        self.penalty_weight = -1.5
        self.plus_weight = 1.5

    def call(self, features):
        user_id = features["user_id"]
        liked_tags = features["liked_tags"]
        unliked_tags = features["unliked_tags"]

        # Embed user ID
        user_id_embedding = self.user_id_model(user_id)
        #user_id_embedding = tf.reduce_mean(user_id_embedding, axis=1)

        # Embed liked and unliked tags
        liked_tags_embedding = self.liked_tags_model(liked_tags)
        unliked_tags_embedding = self.unliked_tags_model(unliked_tags)

        # Apply penalty to unliked tags
        unliked_tags_embedding = unliked_tags_embedding * self.penalty_weight
        liked_tags_embedding = liked_tags_embedding * self.plus_weight


        # Aggregate embeddings
        liked_tags_embedding = tf.reduce_max(liked_tags_embedding, axis=1, keepdims=True)
        unliked_tags_embedding = tf.reduce_min(unliked_tags_embedding, axis=1, keepdims=True)


        # Используйте weighted суммирование для учета штрафов
        user_embedding = tf.add(user_id_embedding, liked_tags_embedding)
        user_embedding = tf.add(user_embedding, unliked_tags_embedding)

        return user_embedding



In [44]:
# Извлечение данных пользователя
test_user = {}
for data in ratings.take(1).as_numpy_iterator():
    test_user = data
    
user_model = UserModel(unique_user_ids, tags_model, embedding_dimension)

user_model(test_user)

<tf.Tensor: shape=(30, 32), dtype=float32, numpy=
array([[-0.03976297, -0.04187637,  0.02398592,  0.02209097,  0.00898775,
         0.03963254,  0.02183858,  0.03877072, -0.04073378,  0.01161937,
         0.01351625, -0.04927077, -0.0445059 , -0.00558278, -0.01821656,
        -0.02406272,  0.00990583,  0.04035266,  0.04487851,  0.02684188,
         0.01571327,  0.02949226,  0.01004782,  0.01906581,  0.03167447,
        -0.02848922, -0.03158103,  0.00759529,  0.03222987,  0.02969871,
        -0.01239014,  0.03494646],
       [-0.04147175, -0.04358515,  0.02227715,  0.0203822 ,  0.00727897,
         0.03792375,  0.0201298 ,  0.03706193, -0.04244257,  0.00991058,
         0.01180746, -0.05097955, -0.04621468, -0.00729156, -0.01992534,
        -0.02577151,  0.00819705,  0.03864388,  0.04316974,  0.02513309,
         0.01400448,  0.02778347,  0.00833904,  0.01735704,  0.0299657 ,
        -0.03019801, -0.03328981,  0.0058865 ,  0.03052109,  0.02798992,
        -0.01409893,  0.03323768],
    

In [52]:
class RecipeAndUserModel(tfrs.Model):

    def _reduce_mean_if_needed(self, embedding):
        """Сокращает размерность эмбеддинга до среднего, если он многомерный."""
        if len(embedding.shape) >= 3:
            return tf.reduce_mean(embedding, axis=1)
        return embedding

    def __init__(self, unique_recipe_names, unique_tags, embedding_dimension):
        super().__init__()
        self.embedding_dimension = embedding_dimension
        # Модель пользователя
        self.user_model = UserModel(unique_user_ids, tags_model, 64)

        # Recipe embeddings
        self.recipe_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_recipe_names, mask_token=None),
            tf.keras.layers.Embedding(len(unique_recipe_names) + 1, embedding_dimension)
        ])

        # Tags embeddings
        self.tags_model = tags_model

        self.dense = tf.keras.layers.Dense(embedding_dimension)  # Сжимаем до размерности 32

        # Подготовка кандидатов, учитывающих и теги
        self.candidates = recipes.batch(128).map(self._get_candidate_embeddings)

        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=self.candidates
            )
        )
        
    def _get_candidate_embeddings(self, recipe_name):
        # Получаем эмбеддинги для имен и тегов
        recipe_embedding = self.recipe_model(recipe_name['recipe_name'])
        print(recipe_name['tags'])
        tags_embedding = self.tags_model(recipe_name['tags'])  # Здесь предполагается, что recipe_name может содержать теги
        recipe_embedding = self._reduce_mean_if_needed(recipe_embedding)
        tags_embedding = self._reduce_mean_if_needed(tags_embedding)
        # Объединяем эмбеддинги имен и тегов
        combined_embedding = tf.concat([recipe_embedding, tags_embedding], axis=1)
        return combined_embedding

    def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        # User embedding
        user_embedding = self.user_model(features)

        # Recipe name embeddings
        recipe_name_embeddings = self.recipe_model(features["recipe_name"])
        #recipe_name_embeddings = self._reduce_mean_if_needed(recipe_name_embeddings)
        # Tags embeddings
        tags = features["tags"]
        tags_embeddings = self.tags_model(tags)
        tags_embeddings = self._reduce_mean_if_needed(tags_embeddings)
        # # Вычисление средневзвешенного значения для тегов (можно модифицировать веса)
        # tags_embeddings_weighted_sum = tf.reduce_sum(tags_embeddings, axis=1)
        # sum_of_weights = tf.reduce_sum(tf.ones_like(tags_embeddings), axis=1)
        # tags_embeddings = tags_embeddings_weighted_sum / sum_of_weights
        
        # recipe_name_embeddings = self._reduce_mean_if_needed(recipe_name_embeddings)
        # tags_embeddings = tf.expand_dims(tags_embeddings, axis=0)
        # Конкатенация эмбеддингов рецепта и тегов
        recipe_embeddings = tf.concat([recipe_name_embeddings, tags_embeddings], axis=1)

        return user_embedding, recipe_embeddings

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        user_embeddings, recipe_embeddings = self(features)
        print("Computed: ", user_embeddings.shape, recipe_embeddings.shape)
        # Используем задачу факторизованного топ-K для оптимизации
        user_embeddings = self._reduce_mean_if_needed(user_embeddings)
        recipe_embeddings = self._reduce_mean_if_needed(recipe_embeddings)
        
        # user_embeddings = tf.keras.layers.Dense(64)(user_embeddings)
        # recipe_embeddings = tf.keras.layers.Dense(64)(recipe_embeddings)
        print("Computed: ", user_embeddings.shape, recipe_embeddings.shape)
        return self.task(user_embeddings, recipe_embeddings)


In [46]:
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(5_000)
test = shuffled.skip(80_000).take(20_000)

cached_train =  train.batch(10_000).cache()
cached_test = test.batch(4096).cache()

In [53]:
model = RecipeAndUserModel(unique_recipe_names, unique_tags, embedding_dimension)
early_callback = tf.keras.callbacks.EarlyStopping(monitor='loss',  patience=5, min_delta=0.001)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
model.fit(cached_train, epochs=12)

Tensor("args_2:0", shape=(None, None), dtype=string)
Tensor("args_2:0", shape=(None, None), dtype=string)
Epoch 1/12
Computed:  (None, None, 64) (None, 64)
Computed:  (None, 64) (None, 64)
Computed:  (None, None, 64) (None, 64)
Computed:  (None, 64) (None, 64)
1/1 [==============================] - 21s 21s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 4.0000e-04 - factorized_top_k/top_100_categorical_accuracy: 0.0010 - loss: 42586.2812 - regularization_loss: 0.0000e+00 - total_loss: 42586.2812
Epoch 2/12
1/1 [==============================] - 20s 20s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top

In [ ]:
model.evaluate(cached_test, return_dict=True)

### Making predictions

In [ ]:
unique_tags[:20]

In [ ]:
likes = np.full(15, "asian")
likes_2 = np.full(15, "african")
likes = np.append(likes,likes_2)
likes

In [ ]:
dislikes = np.full(10, '30-minutes-or-less')
dislikes_2 = np.full(10, '60-minutes-or-less')
dislikes_3 = np.full(10, '15-minutes-or-less')
dislikes = np.append(dislikes, [dislikes_2, dislikes_3])
dislikes

In [ ]:
test_user['liked_tags']

In [ ]:
# Извлечение данных пользователя
test_user = {}
for data in test.take(1).as_numpy_iterator():
    test_user = data

# Создание набора данных для тестового пользователя
user_id_to_test = "42"  # Замените на нужный ID пользователя
test_user_dataset = {
    "user_id": np.array(test_user["user_id"]),
    "liked_tags": np.array(likes),
    "unliked_tags": np.array(dislikes)
}

brute_force = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
def _reduce_mean_if_needed(embedding):
    """Сокращает размерность эмбеддинга до среднего, если он многомерный."""
    if len(embedding.shape) >= 3:
        return tf.reduce_mean(embedding, axis=1)
    return embedding
dense = tf.keras.layers.Dense(96)

def _get_candidate_embeddings(recipe_name):
    # Получаем эмбеддинги для имен и тегов
    recipe_embedding =  model.recipe_model(recipe_name['recipe_name'])
    print(recipe_name['tags'])
    tags_embedding = model.tags_model(recipe_name['tags'])  # Здесь предполагается, что recipe_name может содержать теги
    recipe_embedding = _reduce_mean_if_needed(recipe_embedding)
    tags_embedding = _reduce_mean_if_needed(tags_embedding)
    # Объединяем эмбеддинги имен и тегов
    combined_embedding = tf.concat([recipe_embedding, tags_embedding], axis=1)
    return combined_embedding
    
# Создание набора данных для рецептов и тегов
def get_recipe_and_tag_embeddings(x):
    recipe_embedding = model.recipe_model(x['recipe_name'])  # (None, 32)
    tag_embedding = model.tags_model(x['tags'])  # (None, 32)
    recipe_embedding =_reduce_mean_if_needed(recipe_embedding)
    tag_embedding = _reduce_mean_if_needed(tag_embedding)
    
    combined_embeddings = tf.concat([_reduce_mean_if_needed(recipe_embedding),  _reduce_mean_if_needed(tag_embedding)], axis=1)  # (None, 64)
    
    return dense(combined_embeddings)

# Используем map для обработки всех данных сразу
recipes_embeddings = recipes.batch(100).map(_get_candidate_embeddings)


# Создание набора имен рецептов
recipes_names = recipes.batch(100).map(lambda x: x["recipe_id"])  # Убедитесь, что это (None,)

# Объединяем эмбеддинги с именами
index = brute_force.index_from_dataset(
    tf.data.Dataset.zip((recipes_names, recipes_embeddings))
)

# Получаем рекомендации
_, titles = index(test_user_dataset, k=15)

# Вывод тегов для рекомендуемых рецептов
recommended_tags = raw_recipes[raw_recipes['id'].isin(titles.numpy().flatten().astype('str'))]


In [ ]:
recommended_tags[['name', 'tags_str']]

In [ ]:
recommended_tags[['name', 'tags_str']]